### Imports

In [ ]:
import pandas as pd
import numpy as np
import operator
from itertools import islice

from rdkit import Chem, DataStructs
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors as rdMD

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import recall_score, roc_auc_score

### Helper functions

In [ ]:
def get_morgan_fp(mol):
    """
    Returns the RDKit Morgan fingerprint for a molecule
    """
    info = {}
    fp = rdMD.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024, useFeatures=False, bitInfo=info)
    return fp

def tanimoto(F1,F2):
    """
    Returns tanimoto similarity score for two fingerprints
    """
    Na=float(F1.count("1"))
    Nb=float(F2.count("1"))
    index_Na = [i for i, x in enumerate(F1) if x == "1"]
    index_Nb = [i for i, x in enumerate(F2) if x == "1"]
    pos=list(set(index_Na).intersection(index_Nb))
    Nc=float(len(pos))
    tc=(Nc)/(Na+Nb-Nc)
    tc=round(tc,1)
    return tc

def take(n, iterable):
    """
    Return first n items of the iterable as a list
    """
    return list(islice(iterable, n))

def getNeighbors(x_train, testInstance):
    """
    Returns k neighbors with their similarity scores and activity classes
    """
    similarity = {}
    sorted_similarity = {}
    ac = {}
    k_sim_scores = []
    k_neighbors = []
    for j in range(len(x_train)):
        #sim = tanimoto(testInstance,x_train[j])
        #similarity[(x_train.index[j])] = sim
        dice = DataStructs.DiceSimilarity(testInstance, x_train[j])
        dice = round(dice, 2)
        similarity[(x_train.index[j])] = dice
        ac[(y_train.index[j])] = y_train[j]
    sorted_similarity = sorted(similarity.items(), key=operator.itemgetter(1), reverse=True)
    k_sim_scores = take(k[n], sorted_similarity)
    return ac,dict(k_sim_scores)

def get_prediction(k_sim_scores,ac):
    """
    Returns predicted score and class for a testset compound
    """
    k = len(k_sim_scores)
    maxi = ((k/2)+1)
    active=0
    inactive=0
    pred_class = []
    active_score = []
    inactive_score = []
    pred_score = []
    for key in k_sim_scores:
        if key in ac and ac[key] == 1:
            active += 1
            active_score.append(k_sim_scores[key])
        else:
            inactive += 1
            inactive_score.append(k_sim_scores[key])
    if active >= maxi:
        pred_class.append(1)
        mean_tc = sum(active_score)/len(active_score)
        pred_score.append(round(mean_tc,2))
    else:
        pred_class.append(0)
        mean_tc = sum(inactive_score)/len(inactive_score)
        mean_tc = 1-mean_tc
        pred_score.append(round(mean_tc,2))
    
    return pred_class,pred_score


### Data preparation

In [78]:
# The data file should contain three columns 
# 1. molecule ID;
# 2. canonical SMILES; and 
# 3. activity (which is either 1 or 0)

# reading the data file into a pandas data frame
df=pd.read_csv('/Publications/hERG/datasets/chembl_training_T3.csv', header=0, index_col=0)

# Build ROMol objects 
PandasTools.AddMoleculeColumnToFrame(df, smilesCol='can_smiles')

# Remove molecules that could not be parsed from SMILES
df = df[~df.ROMol.isnull()]

# Calculate fingerprints and store them in df
# Note: if additional fingerprints are needed that are not available in RDkit, they must be imported with the data
df['fp'] = df.apply(lambda x: get_morgan_fp(x['ROMol']), axis=1)
#df.head()

### Defining X and Y

In [79]:
# create X variable (=features i.e. molecular fingerprints)
x = df['fp']
#print(X.shape)

# create Y variable (=activity values i.e. blocker;1 or non-blocker;0)
y = df['ac']
#print(y.shape)

### Cross Validation

In [80]:
# Different k parameter for cross-validation
k=[1]
# if multiple k parameters are needed, then initialize k as k=[1,3,5,10]
for n in range(len(k)):
    
    # Initialize performance measures
    sens     = np.array([])
    spec     = np.array([])
    mean_auc = np.array([])
    
    # 10-fold cross-validation split
    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
    
    for train_index, test_index in sss.split(x,y):
        # Split data to training and test set
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        z += 1
        pred_class = []
        prediction = []
        pred_prob = []
        y_true = []
        
        # Training a k-NN classifier and predicting the test set
        for i in range(len(x_test)):
            ac = {}
            k_sim_scores = {}
            ac,k_sim_scores = getNeighbors(x_train, x_test[i])
            prediction = get_prediction(k_sim_scores, ac)
            pred_class.append(int(prediction[0][0]))
            pred_prob.append(prediction[1][0])
            y_true.append(y_test[i])
            
        # Append performance measures
        sens = np.append(sens, recall_score(y_true, pred_class, pos_label=1))
        spec = np.append(spec, recall_score(y_true, pred_class, pos_label=0))
        mean_auc = np.append(mean_auc, roc_auc_score(y_true, pred_prob))
    
    # 10-fold cross-validation performance
    print('AUC:\t\t%.2f +/- %.2f' % (mean_auc.mean(), mean_auc.std()))
    print('Sensitivity:\t%.2f +/- %.2f' % (sens.mean(), sens.std()))
    print('Specificity:\t%.2f +/- %.2f' % (spec.mean(), spec.std()))

AUC:		0.91 +/- 0.01
Sensitivity:	0.88 +/- 0.02
Specificity:	0.87 +/- 0.01
